In [345]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
from yahooquery import Ticker

In [346]:
year = 2024
df = pd.read_csv(f'Data/sp500_wrds_{year}_fundamental.csv')
price_df = pd.read_csv(f'Data/sp500_wrds_{year}_price.csv') # include 12 months before data
# df.head()

In [347]:
len(df.columns)

76

In [348]:
# initial_columns = ['permno', 'adate', 'qdate', 'public_date', 'bm', 'evm', 'pe_op_basic',
#        'pe_op_dil', 'pe_exi', 'ps', 'pcf', 'dpr', 'npm', 'opmad', 'gpm',
#        'ptpm', 'cfm', 'roa', 'roe', 'roce', 'efftax', 'aftret_eq',
#        'aftret_invcapx', 'aftret_equity', 'pretret_noa', 'GProf',
#        'equity_invcap', 'debt_invcap', 'totdebt_invcap', 'capital_ratio',
#        'int_debt', 'int_totdebt', 'cash_lt', 'invt_act', 'rect_act', 'debt_at',
#        'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt', 'profit_lct',
#        'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets',
#        'debt_capital', 'de_ratio', 'intcov', 'intcov_ratio', 'inv_turn',
#        'at_turn', 'rect_turn', 'pay_turn', 'sale_equity', 'sale_nwc',
#        'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ptb', 'PEG_trailing',
#        'divyield', 'TICKER']
# # some are dropped due to high missing values or redundancy after this block

# df = df[initial_columns]

In [349]:
cols_to_drop = ['permno','adate', 'qdate','PEG_trailing','inv_turn', 'sale_nwc','profit_lct','ocf_lct','curr_debt', 'pretret_noa', 'intcov', 'intcov_ratio', 'rect_act','invt_act']

In [350]:
df = df.drop(columns=cols_to_drop)
df = df.rename(columns={'TICKER': 'ticker', 'public_date': 'date'})
df = df.dropna(subset=['ticker'])

In [351]:
# df.isna().sum()
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
# df.set_index('date', inplace=True)

In [352]:
#remoce % sign from divyield and convert to float
df["divyield"] = df["divyield"].str.rstrip('%')
df["divyield"] = df["divyield"].fillna(0)
df["divyield"] = df["divyield"].astype('float')

In [353]:
df = df.sort_values(['ticker', 'date'])
df = df.groupby('ticker').apply(lambda x: x.ffill().bfill())
df = df.reset_index(drop=True)

In [354]:
df.head()

,gvkey,date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,...,sale_invcap,sale_equity,rd_sale,adv_sale,staff_sale,accrual,ptb,divyield,ticker,cusip
0,126554,2024-01-31,34.692,0.195,19.343,29.908,30.116,31.050,31.050,5.579,...,0.786,1.169,0.070,0.008,0.0,-0.050,6.480,0.726,A,00846U10
1,126554,2024-02-29,36.633,0.195,19.343,31.577,31.796,32.783,32.783,5.891,...,0.786,1.169,0.070,0.008,0.0,-0.050,6.842,0.687,A,00846U10
2,126554,2024-03-31,39.881,0.162,23.184,33.297,33.605,34.645,34.645,6.331,...,0.787,1.153,0.072,0.008,0.0,-0.067,6.891,0.649,A,00846U10
3,126554,2024-04-30,37.559,0.162,23.184,31.359,31.649,32.629,32.629,5.963,...,0.787,1.153,0.072,0.008,0.0,-0.067,6.490,0.689,A,00846U10
4,126554,2024-05-31,35.742,0.162,23.184,29.842,30.118,31.050,31.050,5.674,...,0.787,1.153,0.072,0.008,0.0,-0.067,6.176,0.724,A,00846U10


In [355]:
df.shape

(5438, 62)

2022 specific

In [356]:
# renam fb ticker to meta (2022 change)
df['ticker'] = df['ticker'].replace({'FB': 'META'})

In [357]:
print(len(df["ticker"].unique()))

468


In [358]:
price_df.columns

Index(['tic', 'datadate', 'gvkey', 'ggroup', 'gind', 'gsector', 'cshtrm',
       'prccm'],
      dtype='object')

In [359]:
# create a dictionary to map ticker to gind
gind_mapping = price_df.set_index('tic')['gind'].to_dict()
gsec_mapping = price_df.set_index('tic')['gsector'].to_dict()
df['gind'] = df['ticker'].map(gind_mapping)
df['gsector'] = df['ticker'].map(gsec_mapping)


price_df = price_df[['tic', 'datadate', 'cshtrm', 'prccm']]
price_df = price_df.rename(columns={'tic': 'ticker', 'datadate': 'date', 'cshtrm': 'volume', 'prccm': 'price'}) # rename columns
price_df['date'] = pd.to_datetime(price_df['date'], format='%Y-%m-%d')
# price_df.set_index('date', inplace=True)

In [360]:
# group = price_df.groupby('ticker')
# price_df['ret_1m']  = group['price'].transform(lambda x: x.pct_change())
# price_df['ret_3m']  = group['price'].transform(lambda x: x.pct_change(3))
# price_df['ret_6m']  = group['price'].transform(lambda x: x.pct_change(6))

# # 1. Simple Moving Averages
# price_df['SMA_3']  = group['ret_1m'].transform(lambda x: x.rolling(3).mean())
# price_df['SMA_6']  = group['ret_1m'].transform(lambda x: x.rolling(6).mean())

# # 3. Volatility (Rolling Std of returns)
# price_df['vol_3m']  = group['ret_1m'].transform(lambda x: x.rolling(3).std())
# price_df['vol_6m']  = group['ret_1m'].transform(lambda x: x.rolling(6).std())
# # price_df['vol_12m'] = group['ret_1m'].transform(lambda x: x.rolling(12).std())

# # 4. Exponential Moving Average
# price_df['EMA_6']  = group['ret_1m'].transform(lambda x: x.ewm(span=3, adjust=False).mean())
# price_df['EMA_12'] = group['ret_1m'].transform(lambda x: x.ewm(span=6, adjust=False).mean())


# # 7. Volume moving average
# price_df['vol_SMA_3'] = group['volume'].transform(lambda x: x.rolling(3).mean())
# price_df['vol_SMA_6'] = group['volume'].transform(lambda x: x.rolling(6).mean())

# # 8. Volume Rate of Change
# price_df['vol_ROC'] = group['volume'].transform(lambda x: x.pct_change())

# # 9. OBV (On-Balance Volume)
# def obv(df):
#     sign = df['price'].diff().apply(lambda x: 1 if x>0 else -1 if x<0 else 0)
#     return (sign * df['volume']).cumsum()

# price_df['OBV'] = group.apply(obv).reset_index(level=0, drop=True)


In [361]:
# Assuming price_df is set up
group = price_df.groupby('ticker')

# --- 1. Base Returns (Correct) ---
price_df['ret_1m']  = group['price'].transform(lambda x: x.pct_change())
price_df['ret_3m']  = group['price'].transform(lambda x: x.pct_change(3))
price_df['ret_6m']  = group['price'].transform(lambda x: x.pct_change(6))

# --- 2. SMA of Returns (Correct) ---
price_df['SMA_3']   = group['ret_1m'].transform(lambda x: x.rolling(3).mean())
price_df['SMA_6']   = group['ret_1m'].transform(lambda x: x.rolling(6).mean())

# --- 3. Volatility (Correct) ---
price_df['vol_3m']  = group['ret_1m'].transform(lambda x: x.rolling(3).std())
price_df['vol_6m']  = group['ret_1m'].transform(lambda x: x.rolling(6).std())

# --- 4. EMA of Returns (Fixed parameters) ---
# Fixed: span should match the variable name concept (or close to it)
price_df['EMA_3']   = group['ret_1m'].transform(lambda x: x.ewm(span=3, adjust=False).mean())
price_df['EMA_6']  = group['ret_1m'].transform(lambda x: x.ewm(span=6, adjust=False).mean())

# --- 5. Relative Volume (Fixed Volume SMA) ---
# logic: Current Volume / Average Volume
# If result > 1, volume is higher than average.
price_df['RVOL_3'] = group['volume'].transform(lambda x: x / x.rolling(3).mean())
price_df['RVOL_6'] = group['volume'].transform(lambda x: x / x.rolling(6).mean())

# --- 6. Volume Rate of Change (Correct) ---
price_df['vol_ROC'] = group['volume'].transform(lambda x: x.pct_change())

# --- 7. Volume-Weighted Momentum (Fixed OBV) ---
# Instead of cumulative OBV, we look at the immediate impact of volume on price.
# Logic: Return * Relative Volume. 
# A high return on high volume = Strong Signal.
price_df['vol_mom'] = price_df['ret_1m'] * price_df['RVOL_3']

# Final Cleanup
price_df.dropna(inplace=True)

In [362]:
price_df.isna().sum()

ticker     0
date       0
volume     0
price      0
ret_1m     0
ret_3m     0
ret_6m     0
SMA_3      0
SMA_6      0
vol_3m     0
vol_6m     0
EMA_3      0
EMA_6      0
RVOL_3     0
RVOL_6     0
vol_ROC    0
vol_mom    0
dtype: int64

In [363]:
# import pandas as pd
# import numpy as np


# # -------------------------------------------------------------------------
# # 2. CALCULATIONS
# # -------------------------------------------------------------------------
# group = price_df.groupby('ticker')

# # --- A. Base Returns (Momentum) ---
# # Pure percentage change over time windows
# price_df['ret_1m']  = group['price'].transform(lambda x: x.pct_change())
# price_df['ret_3m']  = group['price'].transform(lambda x: x.pct_change(3))
# price_df['ret_6m']  = group['price'].transform(lambda x: x.pct_change(6))
# # price_df['ret_12m'] = group['price'].transform(lambda x: x.pct_change(12))

# # --- B. Return Moving Averages (Trend of Momentum) ---
# # Averaging the *returns*, not the price, to keep it scale-independent
# price_df['SMA_3_ret']  = group['ret_1m'].transform(lambda x: x.rolling(3).mean())
# price_df['SMA_6_ret']  = group['ret_1m'].transform(lambda x: x.rolling(6).mean())

# # --- C. Volatility (Risk) ---
# # Rolling standard deviation of returns
# price_df['vol_3m']  = group['ret_1m'].transform(lambda x: x.rolling(3).std())
# price_df['vol_6m']  = group['ret_1m'].transform(lambda x: x.rolling(6).std())
# price_df['vol_12m'] = group['ret_1m'].transform(lambda x: x.rolling(12).std())

# # --- D. Volatility Ratio (Regime Change) ---
# # Compares short-term vol to long-term vol (>1 = expanding volatility)
# price_df['vol_ratio'] = price_df['vol_3m'] / price_df['vol_12m']

# # --- E. Distance from Moving Average (Trend Oscillator) ---
# # Percentage distance from the 12-month average price
# price_df['dist_SMA_12'] = group['price'].transform(
#     lambda x: (x - x.rolling(12).mean()) / x.rolling(12).mean()
# )

# # --- F. Distance from High (Drawdown) ---
# # How far is current price from the 12-month maximum? (0 = at high, -0.2 = 20% down)
# price_df['dist_high_12'] = group['price'].transform(
#     lambda x: (x / x.rolling(12).max()) - 1
# )

# # --- G. Bollinger %B / Z-Score (Statistical Position) ---
# # Number of standard deviations the price is from the 12-month mean
# def z_score_price(x, window=12):
#     r_mean = x.rolling(window).mean()
#     r_std = x.rolling(window).std()
#     return (x - r_mean) / r_std

# price_df['price_zscore_12'] = group['price'].transform(z_score_price)

# # --- H. RSI (Relative Strength Index) ---
# # Standard RSI is naturally 0-100 and scale independent
# def calculate_rsi(series, period=14):
#     delta = series.diff()
#     gain = delta.clip(lower=0).rolling(period).mean()
#     loss = -delta.clip(upper=0).rolling(period).mean()
#     rs = gain / loss.replace(0, np.nan) 
#     return 100 - (100 / (1 + rs))

# price_df['RSI_14'] = group['price'].transform(calculate_rsi)

# # --- I. Normalized MACD (Percentage Price Oscillator) ---
# # (Fast EMA - Slow EMA) divided by Price to get a percentage
# def normalized_macd(x):
#     exp1 = x.ewm(span=12, adjust=False).mean()
#     exp2 = x.ewm(span=26, adjust=False).mean()
#     return (exp1 - exp2) / x 

# price_df['MACD_norm'] = group['price'].transform(normalized_macd)

# # --- J. Volume Indicators (Normalized) ---

# # 1. Volume Rate of Change
# price_df['vol_ROC'] = group['volume'].transform(lambda x: x.pct_change())

# # 2. Relative Volume (RVOL) - Ratio of current vol to 10-month avg
# def relative_volume(x, period=10):
#     return x / x.rolling(period).mean()

# price_df['RVOL_10'] = group['volume'].transform(relative_volume)

# # 3. Volume Weighted Momentum
# # Combines price strength with volume conviction
# price_df['vol_wgt_mom'] = price_df['ret_1m'] * price_df['RVOL_10']

# # -------------------------------------------------------------------------
# # 3. CLEANUP
# # -------------------------------------------------------------------------

# # Because we used rolling windows (up to 12 months), the first 12 rows 
# # per ticker will have NaNs. 
# # price_df_clean = price_df.dropna()

In [364]:
price_df.columns

Index(['ticker', 'date', 'volume', 'price', 'ret_1m', 'ret_3m', 'ret_6m',
       'SMA_3', 'SMA_6', 'vol_3m', 'vol_6m', 'EMA_3', 'EMA_6', 'RVOL_3',
       'RVOL_6', 'vol_ROC', 'vol_mom'],
      dtype='object')

In [365]:
price_df.shape

(9233, 17)

In [366]:
nans_col = df.isna().sum()
missing_columns = nans_col[nans_col > 0]

for col in missing_columns.index:
    if col not in ['gind', 'gsector']:
        # 1. Fill using industry mean
        df[col] = df[col].fillna(df.groupby('gind')[col].transform('mean'))
        # 2. Fill remaining using sector mean
        df[col] = df[col].fillna(df.groupby('gsector')[col].transform('mean'))
        

In [367]:
# df.isna().sum()

In [368]:
# df.isna().sum()

In [369]:
# all other missing values imputed using KNN imputer
from sklearn.impute import KNNImputer

num_cols = df.select_dtypes(include='number').columns
imputer = KNNImputer(n_neighbors=3)
df[num_cols] = imputer.fit_transform(df[num_cols])

# still check for any missing values then simple fillna with column mean for numerical columns
df.drop(['gind', 'gsector'], axis=1, inplace=True)
num_cols = df.select_dtypes(include='number').columns
for col in num_cols:
    df[col] = df[col].fillna(df[col].mean())

In [370]:
# df.isna().sum()

In [371]:
price_tickers = price_df['ticker'].unique()
fundamentals_tickers = df['ticker'].unique()
common_tickers = set(price_tickers).intersection(set(fundamentals_tickers))
print(f"Number of unique tickers in fundamentals data: {len(fundamentals_tickers)}")
print(f"Number of unique tickers in price data: {len(price_tickers)}")
print(f"Number of common tickers: {len(common_tickers)}")

Number of unique tickers in fundamentals data: 468
Number of unique tickers in price data: 516
Number of common tickers: 453


In [372]:
# Need to manually check which tickers are missing in either dataset
missing_tickers = set(fundamentals_tickers) - common_tickers
print(f"Tickers in fundamentals data but missing in price data: {len(missing_tickers)}")

Tickers in fundamentals data but missing in price data: 15


In [373]:
price_df.shape, df.shape

((9233, 17), (5438, 62))

In [374]:
df = df.set_index(['date', 'ticker'])
price_df = price_df.set_index(['date', 'ticker'])
df_merged = df.join(price_df, how='inner')
df_merged.reset_index(inplace=True)

In [375]:
df_merged.shape, len(df_merged['ticker'].unique())

((4962, 77), 452)

In [376]:
# df_merged.isna().sum()

In [377]:
# df_merged.isna().sum()

In [378]:
# show tickers with a single missing values
missing_tickers = df_merged[df_merged.isna().any(axis=1)]['ticker'].unique()
missing_tickers # investigate these tickers

array([], dtype=object)

In [379]:
df.isna().sum()

gvkey              0
CAPEI              0
bm                 0
evm                0
pe_op_basic        0
pe_op_dil          0
pe_exi             0
pe_inc             0
ps                 0
pcf                0
dpr                0
npm                0
opmbd              0
opmad              0
gpm                0
ptpm               0
cfm                0
roa                0
roe                0
roce               0
efftax             0
aftret_eq          0
aftret_invcapx     0
aftret_equity      0
pretret_earnat     0
GProf              0
equity_invcap      0
debt_invcap        0
totdebt_invcap     0
capital_ratio      0
int_debt           0
int_totdebt        0
cash_lt            0
debt_at            0
debt_ebitda        0
short_debt         0
lt_debt            0
cash_debt          0
fcf_ocf            0
lt_ppent           0
dltt_be            0
debt_assets        0
debt_capital       0
de_ratio           0
cash_ratio         0
quick_ratio        0
curr_ratio         0
cash_conversi

In [380]:
# droping these tickers
df_merged = df_merged[~df_merged['ticker'].isin(missing_tickers)]
df_merged.isna().sum().sum()

0

In [381]:
# if above is not 0, see why

In [382]:
# save csv to be used in modeling
df_merged.to_csv(f'Data/sp500_cleaned_{year}.csv', index=False)

In [383]:
df_merged.shape

(4962, 77)

In [384]:
print(df_merged["ticker"].nunique())
print(year)

452
2024


In [385]:
# df_merged.columns

In [386]:
df_merged[price_df.columns].isna().sum()

volume     0
price      0
ret_1m     0
ret_3m     0
ret_6m     0
SMA_3      0
SMA_6      0
vol_3m     0
vol_6m     0
EMA_3      0
EMA_6      0
RVOL_3     0
RVOL_6     0
vol_ROC    0
vol_mom    0
dtype: int64